In [1]:
# !pip install graphviz

In [2]:
# Data Processing
import pandas as pd
import sklearn as sk
import time as t  
from functools import reduce
from datetime import date, datetime, timedelta
from joblib import Parallel, delayed
from functools import reduce  # For Python 3.x
import numpy as np
from math import modf
import sqlite3
from delta import * 
import os
from zlib import crc32
from pyspark.sql.types import StringType


# Modelling
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn import preprocessing

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
# import graphviz

#Pyspark data handling
import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

In [3]:
conf = (SparkConf().set("spark.driver.maxResultSize", "4g"))

# Create new context
sc = SparkContext(conf=conf)


# sc = SparkContext("local", "Simple App")

# Create SparkSession 
spark = SparkSession.builder.master("local[1]")\
                        .appName("SparkByExamples.com")\
                        .getOrCreate() 
spark.read.option("header",True) \
          .csv("PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")
df_population = spark.sql("SELECT * FROM population")

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/07 16:42:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/07 16:42:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
calculated_features = [    
          "calendar_day", 
          "f_1_dtc38_1th_15d", 
          "f_1_dtc38_2nd_15d", 
          "f_2_dtc38_1th_15d", 
          "f_2_dtc38_2nd_15d", 
          "f_3_dtc38_1th_15d", 
          "f_3_dtc38_2nd_15d", 
          "f_4_dtc38_1th_15d", 
          "f_4_dtc38_2nd_15d", 
          "f_5_dtc38_1th_15d", 
          "f_5_dtc38_2nd_15d", 
          "f_6_dtc38_1th_15d", 
          "f_6_dtc38_2nd_15d", 
          "f_7_dtc38_1th_15d", 
          "f_7_dtc38_2nd_15d", 
          "f_8_dtc38_1th_15d", 
          "f_8_dtc38_2nd_15d", 

          "f_1_dtc75_1th_15d", 
          "f_1_dtc75_2nd_15d", 
          "f_2_dtc75_1th_15d", 
          "f_2_dtc75_2nd_15d", 
          "f_3_dtc75_1th_15d", 
          "f_3_dtc75_2nd_15d", 
          "f_4_dtc75_1th_15d", 
          "f_4_dtc75_2nd_15d", 
          "f_5_dtc75_1th_15d", 
          "f_5_dtc75_2nd_15d", 
          "f_6_dtc75_1th_15d", 
          "f_6_dtc75_2nd_15d", 
          "f_7_dtc75_1th_15d", 
          "f_7_dtc75_2nd_15d", 
          "f_8_dtc75_1th_15d", 
          "f_8_dtc75_2nd_15d",


          "f_1_dtc77_1th_15d", 
          "f_1_dtc77_2nd_15d", 
          "f_2_dtc77_1th_15d", 
          "f_2_dtc77_2nd_15d", 
          "f_3_dtc77_1th_15d", 
          "f_3_dtc77_2nd_15d", 
          "f_4_dtc77_1th_15d", 
          "f_4_dtc77_2nd_15d", 
          "f_5_dtc77_1th_15d", 
          "f_5_dtc77_2nd_15d", 
          "f_6_dtc77_1th_15d", 
          "f_6_dtc77_2nd_15d", 
          "f_7_dtc77_1th_15d", 
          "f_7_dtc77_2nd_15d", 
          "f_8_dtc77_1th_15d", 
          "f_8_dtc77_2nd_15d",


          "f_1_dtc86_1th_15d", 
          "f_1_dtc86_2nd_15d", 
          "f_2_dtc86_1th_15d", 
          "f_2_dtc86_2nd_15d", 
          "f_3_dtc86_1th_15d", 
          "f_3_dtc86_2nd_15d", 
          "f_4_dtc86_1th_15d", 
          "f_4_dtc86_2nd_15d", 
          "f_5_dtc86_1th_15d", 
          "f_5_dtc86_2nd_15d", 
          "f_6_dtc86_1th_15d", 
          "f_6_dtc86_2nd_15d", 
          "f_7_dtc86_1th_15d", 
          "f_7_dtc86_2nd_15d", 
          "f_8_dtc86_1th_15d", 
          "f_8_dtc86_2nd_15d",


          "f_1_dtc92_1th_15d", 
          "f_1_dtc92_2nd_15d", 
          "f_2_dtc92_1th_15d", 
          "f_2_dtc92_2nd_15d", 
          "f_3_dtc92_1th_15d", 
          "f_3_dtc92_2nd_15d", 
          "f_4_dtc92_1th_15d", 
          "f_4_dtc92_2nd_15d", 
          "f_5_dtc92_1th_15d", 
          "f_5_dtc92_2nd_15d", 
          "f_6_dtc92_1th_15d", 
          "f_6_dtc92_2nd_15d", 
          "f_7_dtc92_1th_15d", 
          "f_7_dtc92_2nd_15d", 
          "f_8_dtc92_1th_15d", 
          "f_8_dtc92_2nd_15d",


          "f_1_dtc94_1th_15d", 
          "f_1_dtc94_2nd_15d", 
          "f_2_dtc94_1th_15d", 
          "f_2_dtc94_2nd_15d", 
          "f_3_dtc94_1th_15d", 
          "f_3_dtc94_2nd_15d", 
          "f_4_dtc94_1th_15d", 
          "f_4_dtc94_2nd_15d", 
          "f_5_dtc94_1th_15d", 
          "f_5_dtc94_2nd_15d", 
          "f_6_dtc94_1th_15d", 
          "f_6_dtc94_2nd_15d", 
          "f_7_dtc94_1th_15d", 
          "f_7_dtc94_2nd_15d", 
          "f_8_dtc94_1th_15d", 
          "f_8_dtc94_2nd_15d",


          "f_1_dtc0401_1th_15d", 
          "f_1_dtc0401_2nd_15d", 
          "f_2_dtc0401_1th_15d", 
          "f_2_dtc0401_2nd_15d", 
          "f_3_dtc0401_1th_15d", 
          "f_3_dtc0401_2nd_15d", 
          "f_4_dtc0401_1th_15d", 
          "f_4_dtc0401_2nd_15d", 
          "f_5_dtc0401_1th_15d", 
          "f_5_dtc0401_2nd_15d", 
          "f_6_dtc0401_1th_15d", 
          "f_6_dtc0401_2nd_15d", 
          "f_7_dtc0401_1th_15d", 
          "f_7_dtc0401_2nd_15d", 
          "f_8_dtc0401_1th_15d", 
          "f_8_dtc0401_2nd_15d",


          "f_1_dtc2457_1th_15d", 
          "f_1_dtc2457_2nd_15d", 
          "f_2_dtc2457_1th_15d", 
          "f_2_dtc2457_2nd_15d", 
          "f_3_dtc2457_1th_15d", 
          "f_3_dtc2457_2nd_15d", 
          "f_4_dtc2457_1th_15d", 
          "f_4_dtc2457_2nd_15d", 
          "f_5_dtc2457_1th_15d", 
          "f_5_dtc2457_2nd_15d", 
          "f_6_dtc2457_1th_15d", 
          "f_6_dtc2457_2nd_15d", 
          "f_7_dtc2457_1th_15d", 
          "f_7_dtc2457_2nd_15d", 
          "f_8_dtc2457_1th_15d", 
          "f_8_dtc2457_2nd_15d",

          "if_parts_replaced_in_1th_15d", 
          "if_parts_replaced_in_2nd_15d"]

In [5]:
'''
Data Normalization Using Z-score Method
The z-score method (often called standardization) transforms the info into distribution with a mean of 0 and a typical deviation of 1. 
Each standardized value is computed by subtracting the mean of the corresponding feature then dividing by the quality deviation.

One standard is to include an epsilon variable that prevents divide by zero. 
Epsilon is an intentional error added to calculations to prevent creating NaN or Inf.
Fact about z-score[column] = 0: mSince the standard deviation is calculated by taking 
the sum of the squared deviations from the mean, a zero standard deviation can only be possible 
when all the values of a variable are the same (all equal to the mean). 
In this case, those variables have no discriminative power so they can be removed from the analysis. 
They cannot improve any classification, clustering or regression task. 
Many implementations will do it for you or throw an error about a matrix calculation.
'''
def normalize_numerical_data(numeric_columns):
   #first 2 columns, ENGIN_HP and calendar_day are not normalized and neither the latest two columns (target calumns), parts_are_replaced columns
    number_of_numeric_features = len(numeric_columns.columns)-2
    epsilon=1e-100
    for column in numeric_columns.iloc[:,2:number_of_numeric_features].columns:
        numeric_columns[column] = (numeric_columns[column] -
                               numeric_columns[column].mean()) / (numeric_columns[column].std()+epsilon)

    return numeric_columns



'''
This converts a bytes string to a float between 0.0 and 1.0. If we are using a unicode string (e.g., in python 3), then we need to encode it. 
This should give the same result on any machine and any version of python (tested on python 2.7 and 3.5).

Note: the & 0xffffffff is here to guarantee an unsigned int result. 
It is needed because depending on the python version crc32(b) may return a signed or unsigned int.:
'''
def bytes_to_float(s):
    return float(crc32(b) & 0xffffffff) / 2**32


def str_to_float(s, encoding="utf-8"):
    return bytes_to_float(s.astype('str').encode(encoding))


convert_str_to_float_udf = udf(str_to_float)


In [6]:
selected_features_from_population = ['VIN','ENGINE_SIZE','ENGINE_HP','VEH_TYPE']+[s for s in df_population.columns if 'KOLA' in s]
all_columns_names = selected_features_from_population + calculated_features
resultedData = pd.read_csv('/storage/home/yqf5148/work/volvoPennState/data/dataset/resultedData.csv', sep=',', names=all_columns_names, header=None, error_bad_lines=False, index_col=False, dtype='unicode')
# display(resultedData.loc[resultedData['calendar_day'].astype('int32')==375].iloc[: , 1:5])
display(resultedData.fillna(0))

TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'

In [ ]:
number_of_features = len(resultedData.columns)-2
y = resultedData.iloc[: , number_of_features:number_of_features+2]
X = resultedData.iloc[: , :number_of_features]

In [7]:
n_data = resultedData

VINs_in_data = n_data["VIN"].drop_duplicates()
VINs_in_data = VINs_in_data.reset_index(drop=True)

for i, thisVIN in VINs_in_data.iteritems():
    print(thisVIN, thisVIN[i], i)
    whole_data_for_thisVIN=n_data.loc[n_data['VIN']==thisVIN]
    display(whole_data_for_thisVIN)
    
    data_size = len(whole_data_for_thisVIN)
  

    numeric_columns = whole_data_for_thisVIN.select_dtypes(exclude='object')
    numeric_columns_header_list = numeric_columns.columns
    display(numeric_columns)

    non_numeric_columns = whole_data_for_thisVIN.select_dtypes(include='object')
    display(non_numeric_columns)
    
    number_of_non_numeric_columns = len(non_numeric_columns)
    
    non_numeric_columns=non_numeric_columns.replace('',np.nan)
    
#     non_numeric_columns.iloc[0:6 , 1:6] = non_numeric_columns.iloc[0:6 , 1:6].applymap(lambda col:(convert_str_to_float_udf(lit(col))) )
#     non_numeric_columns.iloc[0:6 , 1:6] = non_numeric_columns.iloc[0:6 , 1:6].apply()
#     display(non_numeric_columns)
    
    
    train_data=whole_data_for_thisVIN.sample(frac=0.8,random_state=200)
    test_data=whole_data_for_thisVIN.drop(train_data.index)
    
    y_train = train_data.iloc[: , number_of_features:number_of_features+2]
    X_train = train_data.iloc[: , :number_of_features]
    y_test = test_data.iloc[: , number_of_features:number_of_features+2]
    X_test = test_data.iloc[: , :number_of_features]
    
#     X_train = X_train.drop(['VIN','calendar_day'], axis=1)
#     X_test = X_test.drop(['VIN','calendar_day'], axis=1)


4V4WC9EH7MN275768 4 0


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
6,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
7,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11558,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11559,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11560,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11561,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


""
0
1
2
6
7
...
11558
11559
11560
11561


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
2,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
6,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
7,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11558,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11559,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11560,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
11561,4V4WC9EH7MN275768,D13,425,VNM,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


4V4NC9DH4JN896121 V 1


...

4V4NC9EJ8FN913091 8 8


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
40,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
41,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
42,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
43,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
48,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13024,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13025,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13029,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13032,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1


""
40
41
42
43
48
...
13024
13025
13029
13032


,VIN,ENGINE_SIZE,ENGINE_HP,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
40,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
41,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
42,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
43,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
48,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13024,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13025,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13029,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1
13032,4V4NC9EJ8FN913091,D13,500,VNL,WTDF22.5,WTDD22.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,...,0.0,65188.68,0.0,39826.168,0,12,0,12,0,1


IndexError: string index out of range

In [13]:
onehotencoder = OneHotEncoder(handle_unknown='ignore', sparse=False, dtype=float)
# non_numeric_data[non_numeric_data_header_list] = onehotencoder.fit_transform(non_numeric_data[non_numeric_data_header_list])
# display(non_numeric_data[non_numeric_data_header_list])

non_numeric_features_encoded = onehotencoder.fit_transform(non_numeric_data)
display(non_numeric_features_encoded)
print(non_numeric_features_encoded.shape)

categorical_columns = [f'{col}_{cat}' for i, col in enumerate(non_numeric_data.columns) for cat in onehotencoder.categories_[i]]
# categorical_columns

one_hot_features = pd.DataFrame(non_numeric_features_encoded, columns=categorical_columns)
normalized_and_encoded_data = one_hot_features.join(numeric_columns)
display(normalized_and_encoded_data)






#concating numeric dataframe that is normalized now with those non-numeric ones from the resultedData
# normalized_data = pd.concat([non_numeric_data, numeric_columns], axis=1, ignore_index=True)

# #recreating dataframe in order to add column names
# normalized_data=pd.DataFrame(normalized_data.values.tolist(), columns=all_columns_names)
# not_normalized_data = resultedData
# display(non_numeric_data)

# display(normalized_data)
# display(not_normalized_data)

,VIN,ENGINE_SIZE,VEH_TYPE,_KOLA_01X,_KOLA_02X,_KOLA_03X,_KOLA_04X,_KOLA_05X,_KOLA_06X,_KOLA_07X,...,_KOLA_YJX,_KOLA_YKX,_KOLA_YLX,_KOLA_YVX,_KOLA_Z8C,_KOLA_Z9X,_KOLA_ZAX,_KOLA_ZGX,_KOLA_ZIX,_KOLA_ZMX
0,1M1AW01Y9FM007831,D11,CXU,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-ALCO,?,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,?,ULINER,SUSPL-E5,HUBC-BAS,UXMBB,ESP-BAS1
1,1M1AW01Y9FM007831,D11,CXU,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-ALCO,?,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,?,ULINER,SUSPL-E5,HUBC-BAS,UXMBB,ESP-BAS1
2,1M1AW01Y9FM007831,D11,CXU,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-ALCO,?,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,?,ULINER,SUSPL-E5,HUBC-BAS,UXMBB,ESP-BAS1
3,1M1AW01Y9FM007831,D11,CXU,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-ALCO,?,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,?,ULINER,SUSPL-E5,HUBC-BAS,UXMBB,ESP-BAS1
4,1M1AW01Y9FM007831,D11,CXU,WTDF22.5,WTDD22.5,?,WBF-ALCO,WBD-ALCO,?,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,?,ULINER,SUSPL-E5,HUBC-BAS,UXMBB,ESP-BAS1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3080,1M1AN02Y9JM027295,D13,CHU,WTDF24.5,WTDD24.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,UFLD,ULINER,SUSPL-E,HUBC-BAS,XMBB-ST,ESP-BAS1
3081,1M1AN02Y9JM027295,D13,CHU,WTDF24.5,WTDD24.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,UFLD,ULINER,SUSPL-E,HUBC-BAS,XMBB-ST,ESP-BAS1
3082,1M1AN02Y9JM027295,D13,CHU,WTDF24.5,WTDD24.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,UFLD,ULINER,SUSPL-E,HUBC-BAS,XMBB-ST,ESP-BAS1
3083,1M1AN02Y9JM027295,D13,CHU,WTDF24.5,WTDD24.5,UWTDT,WBF-ALCO,WBD-ALCO,UWBT,UWPF,...,RST-AIR,FSS-LEAF,RSS-AIR,RSHABS,UFLD,ULINER,SUSPL-E,HUBC-BAS,XMBB-ST,ESP-BAS1


array([[0., 1., 1., ..., 1., 0., 1.],
       [0., 1., 1., ..., 1., 0., 1.],
       [0., 1., 1., ..., 1., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.]])

(3085, 830)


,VIN_1M1AN02Y9JM027295,VIN_1M1AW01Y9FM007831,ENGINE_SIZE_D11,ENGINE_SIZE_D13,VEH_TYPE_CHU,VEH_TYPE_CXU,_KOLA_01X_WTDF22.5,_KOLA_01X_WTDF24.5,_KOLA_02X_WTDD22.5,_KOLA_02X_WTDD24.5,...,f_5_dtc2457_1th_15d,f_5_dtc2457_2nd_15d,f_6_dtc2457_1th_15d,f_6_dtc2457_2nd_15d,f_7_dtc2457_1th_15d,f_7_dtc2457_2nd_15d,f_8_dtc2457_1th_15d,f_8_dtc2457_2nd_15d,if_parts_replaced_in_1th_15d,if_parts_replaced_in_2nd_15d
0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
4,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3080,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3081,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3082,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3083,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


# Fitting and Evaluating the Model

In [10]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)

# y_pred = rf.predict(X_test)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

# Pandas random sample will also work

train=df.sample(frac=0.8,random_state=200)
test=df.drop(train.index)


X_train,X_test,y_train,y_test = train_test_split(X.index, y, test_size=0.2)
X.iloc[X_train] # return dataframe train


from sklearn.model_selection import KFold
kf = KFold(n_splits=10, random_state=0)
y_hat_all = []
for train_index, test_index in kf.split(X, y):
    reg = RandomForestRegressor(n_estimators=50, random_state=0)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = reg.fit(X_train, y_train)
    y_hat = clf.predict(X_test)
    y_hat_all.append(y_hat)
    
    
row_count = df.shape[0]
split_point = int(row_count*1/5)
test_dat

a, train_data = df[:split_point], df[split_point:]

In [ ]:
# Train based on Train Data for Each VIN

In [ ]:
# Test based on unseen Data for Each VIN

In [ ]:
# Plot figures of Random Forest

In [ ]:
# Plot figures of Gussian Naiive Base

In [ ]:
# Claculate ROC and Evaluate Results

In [5]:
def random_forest(X_train,X_test,y_train,y_test)
    RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    RF.fit(X, y)
    RF.predict(X.iloc[75:,:])
    round(RF.score(X,y), 4)

ValueError: could not convert string to float: '1M1AN02Y7JM027294'